## Load relevant packages

In [138]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import numpy as np


pd.set_option('display.float_format', '{:.2f}'.format)

os.chdir("C:/Users/timur/WB Datasets/")



def window(df, group_keys, agg_field, aggregation, new_col_name):
    # This function performs a similar function to pysparks window function. We want to partition the data by
    # group_keys, and perform an aggregation over agg_field


    # Create a dictionary of aggregations, for example {"amount" : ["sum"], "transaction" : ["count"]}
    agg_dict = {agg_field:[aggregation]}
     
        
    # Partition the data by the group keys and aggregate. 
    gdf = df.groupby(group_keys).agg(agg_dict).reset_index()

    # Rename the columns
    gdf.columns = group_keys + [new_col_name]

    # Merge back to original dataframe
    df = pd.merge(df, gdf, on=group_keys, how="left")

    return df



## Load Data

In [2]:
data = pd.read_csv("exercise_1_credit_card_fraud.csv")

In [3]:
data.sample(n=10)

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
4294412,307,CASH_OUT,133906.27,C293936813,20172.0,0.00,C1455609307,30131.62,164037.89,0,0
352580,17,TRANSFER,1607801.26,C793593944,0.0,0.00,C741698250,5330804.86,6938606.12,0,0
3419787,256,CASH_OUT,9854.14,C1235460748,152789.9,142935.76,C1037263119,16773870.23,16783724.37,0,0
3648191,275,CASH_OUT,116372.62,C782655531,0.0,0.00,C565373541,240639.18,357011.81,0,0
2184732,185,PAYMENT,48392.20,C2046490104,0.0,0.00,M60128646,0.00,0.00,0,0
90845,10,PAYMENT,7294.99,C758829531,49530.0,42235.01,M1169094321,0.00,0.00,0,0
3768915,280,CASH_OUT,206504.09,C138861917,0.0,0.00,C1200797099,251658.27,458162.36,0,0
2905780,228,CASH_IN,300553.74,C886677802,299186.0,599739.74,C1811466657,271994.36,0.00,0,0
6194957,573,PAYMENT,19963.64,C159152799,374331.0,354367.36,M1705722206,0.00,0.00,0,0
5476800,379,CASH_OUT,3998.73,C392514117,30.0,0.00,C1224026773,30984.51,34983.25,0,0


## Check the proportion of null rows for each field

In [4]:
prop_null = data.isnull().sum().to_frame()/len(data)
prop_null.columns = ["proportionOfRowsNull"]
prop_null

,proportionOfRowsNull
step,0.0
type,0.0
amount,0.0
nameOrig,0.0
oldbalanceOrg,0.0
newbalanceOrig,0.0
nameDest,0.0
oldbalanceDest,0.0
newbalanceDest,0.0
isFraud,0.0


## Remove columns that are all null and duplicate rows

In [5]:
cols_to_drop = prop_null[prop_null["proportionOfRowsNull"]==1].index.tolist()
data = data.drop(columns = cols_to_drop)

# If there was a transactionID column we could drop duplicates however there is no such column data.drop_duplicates(subset=["transactionId"])
# and we can have transactions that look like duplicates as a feature of the data, for example someone sends the same amount of funds to the same 
# entity within the same time period. Although the balances should change across these two transcations we could have offsetting transctions in
# between them. 

 

## Add additional fields to dataset

In [18]:
# Add transaction id columns
data["transactionId"] = data.index

# Calculate the change in balance for the sender and receiver
data["balanceDiffOrig"] = data["newbalanceOrig"] - data["oldbalanceOrg"]
data["balanceDiffDest"] = data["newbalanceDest"] - data["oldbalanceDest"]

# Although this is a simulation from a agent based model, we can assume that we have initial date to calculate the running date from the time steps
INITIAL_DATE = '2023-11-01'
data['dateTime']=pd.to_datetime(data['step'], unit='h', origin=pd.Timestamp(INITIAL_DATE))
data['date'] = data['dateTime'].dt.date
data['dayOfWeek'] = data['dateTime'].dt.dayofweek
data['hour'] = data['dateTime'].dt.hour
data['month'] = data['dateTime'].dt.month

# The initial character from nameOrig and nameDest indicates the entity type (as per discussion here https://www.kaggle.com/datasets/ealaxi/paysim1/discussion/32786)
data["origType"] = data["nameOrig"].apply(lambda x: "Client" if x[0]=="C" else "Merchant")
data["destType"] = data["nameDest"].apply(lambda x: "Client" if x[0]=="C" else "Merchant")
data.sample(n=10)

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,...,transactionId,balanceDiffOrig,balanceDiffDest,dateTime,date,dayOfWeek,hour,month,origType,destType
3918576,284,CASH_OUT,120455.61,C613398506,0.00,0.00,C1865057034,203199.44,323655.04,0,...,3918576,0.00,120455.60,2023-11-12 20:00:00,2023-11-12,6,20,11,Client,Client
4598320,328,PAYMENT,34309.86,C1842595455,0.00,0.00,M820835447,0.00,0.00,0,...,4598320,0.00,0.00,2023-11-14 16:00:00,2023-11-14,1,16,11,Client,Merchant
4325833,308,CASH_IN,386364.86,C861472169,6160248.49,6546613.35,C1709658993,2106490.86,1720126.00,0,...,4325833,386364.86,-386364.86,2023-11-13 20:00:00,2023-11-13,0,20,11,Client,Client
6266861,617,TRANSFER,578826.97,C1708740731,0.00,0.00,C1165812079,1009478.91,1588305.88,0,...,6266861,0.00,578826.97,2023-11-26 17:00:00,2023-11-26,6,17,11,Client,Client
6036774,478,PAYMENT,366.88,C774853099,323110.00,322743.12,M137658216,0.00,0.00,0,...,6036774,-366.88,0.00,2023-11-20 22:00:00,2023-11-20,0,22,11,Client,Merchant
1445048,140,TRANSFER,776377.01,C1686121657,61484.00,0.00,C422040971,119578.12,895955.13,0,...,1445048,-61484.00,776377.01,2023-11-06 20:00:00,2023-11-06,0,20,11,Client,Client
1037310,94,PAYMENT,20337.12,C1951009160,0.00,0.00,M1130112287,0.00,0.00,0,...,1037310,0.00,0.00,2023-11-04 22:00:00,2023-11-04,5,22,11,Client,Merchant
1866101,164,PAYMENT,926.67,C767700939,12295.00,11368.33,M1605693386,0.00,0.00,0,...,1866101,-926.67,0.00,2023-11-07 20:00:00,2023-11-07,1,20,11,Client,Merchant
3538269,260,CASH_OUT,193575.15,C184731686,9959.00,0.00,C1713650357,3848709.11,4042284.26,0,...,3538269,-9959.00,193575.15,2023-11-11 20:00:00,2023-11-11,5,20,11,Client,Client
2293903,187,CASH_OUT,38218.27,C1177445895,49926.00,11707.73,C1534422465,0.00,38218.27,0,...,2293903,-38218.27,38218.27,2023-11-08 19:00:00,2023-11-08,2,19,11,Client,Client


## Data quality checks
Reading the thesis found here https://bth.diva-portal.org/smash/record.jsf?pid=diva2%3A955852&dswid=-4905 I have found the following definitions:

1. CASH-IN is the process of increasing the balance of account by paying in cash to a merchant.
2. CASH-OUT is the opposite process of CASH-IN, it means to withdraw cash from a merchant which decreases the balance of the account.
3. DEBIT is similar process than CASH-OUT and involves sending the money from the mobile money service to a bank account.
4. PAYMENT is the process of paying for goods or services to merchants which decreases the balance of the account and increases the balance of the receiver.
5. TRANSFER is the process of sending money to another user of the service through the mobile money platform.

From these definitions we can come up with these data quality checks

1. TRANSFER must be client to client = transferCheck
2. CASH-OUT must be between client and merchant = cashOutCheck
3. CASH-IN must be between client and merchant = cashinCheck
4. PAYMENT must be between client and merchant = paymentCheck

Other accounting consistency check is 
1. The difference in the senders balance must be equal and the opposite sign of the recievers balance = balanceCheck
2. Amount must be a float = amountCheck


In [7]:
# Summarise transaction type by sender/receiver type
entityTypeByTransactionType = data.groupby(["type", "origType", "destType"])["transactionId"].count().reset_index()
entityTypeByTransactionType.columns = ["type", "origType", "destType", "transactionCount"]
entityTypeByTransactionType

,type,origType,destType,transactionCount
0,CASH_IN,Client,Client,1399284
1,CASH_OUT,Client,Client,2237500
2,DEBIT,Client,Client,41432
3,PAYMENT,Client,Merchant,2151495
4,TRANSFER,Client,Client,532909


The summary table does not align with the relationships we expect

In [16]:
transaction_type_balance_diff_summary = data.groupby(["type"]).agg({ "balanceDiffOrig":[ "sum", "mean"], "balanceDiffDest":[ "sum", "mean"]}).reset_index()
transaction_type_balance_diff_summary .columns = ["type", "balanceDiffOrigSum","balanceDiffOrigMean", "balanceDiffDestSum", "balanceDiffDestMean"]
transaction_type_balance_diff_summary .style.format(precision=2, thousands=",")  

,type,balanceDiffOrigSum,balanceDiffOrigMean,balanceDiffDestSum,balanceDiffDestMean
0,CASH_IN,"236,360,342,005.15","168,915.20","-169,052,273,359.83","-120,813.41"
1,CASH_OUT,"-63,879,756,978.47","-28,549.61","433,108,804,483.48","193,568.18"
2,DEBIT,"-144,418,907.68","-3,485.69","823,158,411.87","19,867.70"
3,PAYMENT,"-13,724,250,333.70","-6,378.94",0.00,0.00
4,TRANSFER,"-23,529,901,460.21","-44,153.70","525,960,456,160.79","986,961.11"


The change in balance between the sender and receiver is not the same in absolute value as we expect. We can apply filters to remove these rows, but we won't be left with any
data so in this case we won't apply them. 

In [120]:
applyChecks = False
if applyChecks:
    # transferCheck
    data = data[ (data["type"] != "Transfer") | (( data["origType"] == "Client") & (data["destType"] == "Client"))]
    # balanceCheck
    data = data[ (data["balanceDiffOrig"]/data["balanceDiffDest"]) == -1]
    # cashOutCheck
    data = data[ (data["type"] != "CASH_OUT") | (( data["origType"] == "Client") & (data["destType"] == "Merchant"))]
    # cashinCheck
    data = data[ (data["type"] != "CASH_IN") | (( data["origType"] == "Client") & (data["destType"] == "Merchant"))]
    # paymentCheck
    data = data[ (data["type"] != "Payement") | (( data["origType"] == "Client") & (data["destType"] == "Merchant"))]
    # amountCheck
    data = data[data["amount"].apply(lambda x: type(x)==float)]

## Summarise transactions by type and whether it is fraudulent or not

In [17]:
transaction_type_summary = data.groupby(["type"]).agg({"amount": ["sum", "mean", "count"]}).reset_index()
transaction_type_summary.columns = ["type", "totalAmount", "averageAmount", "count"]
transaction_type_summary.style.format(precision=2, thousands=",") 

,type,totalAmount,averageAmount,count
0,CASH_IN,"236,367,391,912.46","168,920.24","1,399,284"
1,CASH_OUT,"394,412,995,224.49","176,273.96","2,237,500"
2,DEBIT,"227,199,221.28","5,483.67","41,432"
3,PAYMENT,"28,093,371,138.37","13,057.60","2,151,495"
4,TRANSFER,"485,291,987,263.17","910,647.01","532,909"


The platform is primarily used to transfer funds and cashing out. Though the most frequent transaction type is making payments to merchants

In [91]:
fraud_summary = data.groupby(["isFraud"]).agg({"amount": ["sum", "mean", "count"], "balanceDiffOrig" : ["mean"], "hour" :["mean"]}).reset_index()
fraud_summary.columns = ["isFraud", "totalAmount", "averageAmount", "totalCount", "origAverageBalancChange", "averageHourOfDay"]
fraud_summary.insert(2, "fractionOfTotalValue", fraud_summary["totalAmount"]/fraud_summary["totalAmount"].sum())
fraud_summary.insert(4, "fractionOfTottalTransactions", fraud_summary["totalCount"]/fraud_summary["totalCount"].sum())
fraud_summary.style.format(precision=3, thousands=",") 

,isFraud,totalAmount,fractionOfTotalValue,averageAmount,fractionOfTottalTransactions,totalCount,origAverageBalancChange,averageHourOfDay
0,0,"1,132,336,529,331.930",0.989,"178,197.042",0.999,"6,354,407","23,141.516",15.326
1,1,"12,056,415,427.840",0.011,"1,467,967.299",0.001,"8,213","-1,457,274.974",11.546


Fradulent transactions only consist of around 11% of overall transaction volume, and %0.1 of total transactio count. We do see that fraudulent transactions are on average 8 time larger than non fraudulent transactions

In [78]:
fraud_vs_transaction_type = data.groupby(["isFraud", "type"])["transactionId"].count().reset_index()
fraud_vs_transaction_type.columns = ["isFraud", "type", "count"]
is_fraud_total_count = fraud_summary[["isFraud", "totalCount"]]
fraud_vs_transaction_type = pd.merge(fraud_vs_transaction_type, is_fraud_total_count, on="isFraud")
fraud_vs_transaction_type["proportionOfTransactions"] = fraud_vs_transaction_type["count"]/fraud_vs_transaction_type["totalCount"]
fraud_vs_transaction_type.style.format(precision=4, thousands=",") 

,isFraud,type,count,totalCount,proportionOfTransactions
0,0,CASH_IN,"1,399,284","6,354,407",0.2202
1,0,CASH_OUT,"2,233,384","6,354,407",0.3515
2,0,DEBIT,"41,432","6,354,407",0.0065
3,0,PAYMENT,"2,151,495","6,354,407",0.3386
4,0,TRANSFER,"528,812","6,354,407",0.0832
5,1,CASH_OUT,"4,116","8,213",0.5012
6,1,TRANSFER,"4,097","8,213",0.4988


We see that fraudulent transactions only consist of transfers and cash out, with almost identical frequencies. This indicates that the fraudsters most like transfer the funds and then cash it out. 

Do the fraudsters totally deplete the accounts? To test this
I will look the average new balance following a transfer accross the fradulent and non-fraudulent transactions

In [82]:
new_balance_summary = data[data["type"] == "TRANSFER"]
new_balance_summary.insert(0,"balanceIsDepleted", (new_balance_summary["oldbalanceOrg"]>0) & (new_balance_summary["newbalanceOrig"]==0))
new_balance_summary.groupby(["isFraud"])["balanceIsDepleted"].mean().reset_index()

,isFraud,balanceIsDepleted
0,0,0.42
1,1,0.96


We see that 96% of fradulent transfers deplete the balance of the account, more than double the proportion of non-fraudulent transactions.

Since fraudulent transfers deplete the account, and the conduct an equal amount of cash outs as they do transfers the next question is how quickly to the cash out after transfering the funds. 

## Calculate the date difference between the transfer and cash out across fraudulent and non-fraudulent transactions

If a fraudster is to cash out, then he/she would transfer funds to a recipient and the recipient would then cash out. The recipient of the transfer should appear as the originator in the cash out. Below we see that only 3 of the recipients in the transfer are originators in all cash outs, and 0 that have been deemed to be fraudulent.  

This is not what we expect of the data. 

In [106]:
# Split transaction by transfers and cash_outs
transfers = data[data["type"]=="TRANSFER"]
cash_outs = data[data["type"]=="CASH_OUT"]

fraud_transfers = transfers[transfers["isFraud"] == 1]
fraud_cashouts = cash_outs [cash_outs ["isFraud"] == 1]

non_fraud_transfers = transfers[transfers["isFraud"] == 0]
non_fraud_cashouts = cash_outs [cash_outs ["isFraud"] == 0]


print("Number of fraudulent transactions where the destination name in the transfer is the same as the originated name in a fradulent cash out: ",fraud_transfers["nameDest"].isin(fraud_cashouts["nameOrig"]).sum())
print("Number of fraudulent transactions where the destination name in the transfer is the same as the originated name in all cash outs: ",fraud_transfers["nameDest"].isin(cash_outs["nameOrig"]).sum())



print("Number of non-fraudulent transactions where the destination name in the transfer is the same as the originated name in a non-fradulent cash out: ",non_fraud_transfers["nameDest"].isin(non_fraud_cashouts["nameOrig"]).sum())
print("Number of non-fraudulent transactions where the destination name in the transfer is the same as the originated name in all cash outs: ",non_fraud_transfers["nameDest"].isin(cash_outs["nameOrig"]).sum())


Number of fraudulent transactions where the destination name in the transfer is the same as the originated name in a fradulent cash out:  0
Number of fraudulent transactions where the destination name in the transfer is the same as the originated name in all cash outs:  3
Number of non-fraudulent transactions where the destination name in the transfer is the same as the originated name in a non-fradulent cash out:  568
Number of non-fraudulent transactions where the destination name in the transfer is the same as the originated name in all cash outs:  568


In [139]:
# We investigate the above problem further here. 
fradulent_transfers_cash_outs  = pd.concat([fraud_transfers , fraud_cashouts])

fradulent_transfers_cash_outs  = fradulent_transfers_cash_outs[fradulent_transfers_cash_outs  ["amount"]>0]

# We want to only keep pairs that have at least 2 transactions. 
fradulent_transfers_cash_outs   = window(fradulent_transfers_cash_outs  , ["amount", "step"], "transactionId", "count", "transactionCount")

fradulent_transfers_cash_outs  = fradulent_transfers_cash_outs[fradulent_transfers_cash_outs  ["transactionCount"]>1]

fradulent_transfers_cash_outs.sort_values(by=["amount", "step"]).head(n=20)

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,...,balanceDiffOrig,balanceDiffDest,dateTime,date,dayOfWeek,hour,month,origType,destType,transactionCount
1946,351,TRANSFER,63.80,C1293504491,63.80,0.00,C1368130863,0.00,0.00,1,...,-63.80,0.00,2023-11-15 15:00:00,2023-11-15,2,15,11,Client,Client,2
6059,351,CASH_OUT,63.80,C1861878353,63.80,0.00,C958479953,0.00,63.80,1,...,-63.80,63.80,2023-11-15 15:00:00,2023-11-15,2,15,11,Client,Client,2
83,15,TRANSFER,119.00,C1995557473,119.00,0.00,C1480876722,0.00,0.00,1,...,-119.00,0.00,2023-11-01 15:00:00,2023-11-01,2,15,11,Client,Client,2
4183,15,CASH_OUT,119.00,C1584512618,119.00,0.00,C2102058838,4844060.13,4844179.13,1,...,-119.00,119.00,2023-11-01 15:00:00,2023-11-01,2,15,11,Client,Client,2
3599,653,TRANSFER,119.65,C1497766467,119.65,0.00,C543477940,0.00,0.00,1,...,-119.65,0.00,2023-11-28 05:00:00,2023-11-28,1,5,11,Client,Client,2
7704,653,CASH_OUT,119.65,C773613907,119.65,0.00,C1518370196,1183575.58,1183695.22,1,...,-119.65,119.64,2023-11-28 05:00:00,2023-11-28,1,5,11,Client,Client,2
923,162,TRANSFER,151.00,C1172437299,151.00,0.00,C315826176,0.00,0.00,1,...,-151.00,0.00,2023-11-07 18:00:00,2023-11-07,1,18,11,Client,Client,2
5034,162,CASH_OUT,151.00,C790340353,151.00,0.00,C517676411,386163.34,386314.34,1,...,-151.00,151.00,2023-11-07 18:00:00,2023-11-07,1,18,11,Client,Client,2
35,7,TRANSFER,164.00,C1455969984,164.00,0.00,C2119910556,0.00,0.00,1,...,-164.00,0.00,2023-11-01 07:00:00,2023-11-01,2,7,11,Client,Client,2
4134,7,CASH_OUT,164.00,C1173659886,164.00,0.00,C1769947269,4068.00,0.00,1,...,-164.00,-4068.00,2023-11-01 07:00:00,2023-11-01,2,7,11,Client,Client,2


Looking at the above table we see that fraudulent transfers with the same value and step as fraudulent cash outs have totally different parties involved. 

To be able to say something about this matter, I am going to make a very simple assumption. If we observe a fraudulent transaction that is the same value as the cash out, assume the transfer and cashout are linked. Using this methodology we will link multiple cash outs to each transfer. I will only keep the cash out with the smallest step difference between the transfer and cash out.  


In [141]:

transfer_to_cash_out = pd.merge(fraud_transfers, fraud_cashouts, on="amount",  suffixes=('Transfer', 'Cashout'))
transfer_to_cash_out = transfer_to_cash_out[transfer_to_cash_out["dateTimeTransfer"]<=transfer_to_cash_out["dateTimeCashout"]]
transfer_to_cash_out["hourDiff"] = transfer_to_cash_out["stepCashout"] - transfer_to_cash_out["stepTransfer"]
transfer_to_cash_out = transfer_to_cash_out.sort_values(by="hourDiff", ascending=True)
transfer_to_cash_out = transfer_to_cash_out.drop_duplicates(subset=["transactionIdTransfer"], keep="first")
print("Average hourly difference between transfer and cash out: ", transfer_to_cash_out["hourDiff"].mean())

Average hourly difference between transfer and cash out:  0.0


We see that fraudulent transactions rapidly move funds through transfers and subsequent cash outs.  